<a href="https://colab.research.google.com/github/Leonardoup11/JSON/blob/main/Senior_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install tflearn

     |████████████████████████████████| 107 kB 12.1 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=bc4aa91de84ba8e2b11be766a90266ce1ac03e70f2561fe7c370f9e6aee24346
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [20]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import tflearn
import numpy
import tensorflow
import random
import json
import pandas as pd
import pickle
nltk.download('punkt')

url = ("https://raw.githubusercontent.com/Leonardoup11/pedrin/main/intents.json")
intents = pd.read_json(url)
data = intents
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:


model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")




Training Step: 5999  | total loss: 0.12043 | time: 0.033s
| Adam | epoch: 1000 | loss: 0.12043 - acc: 0.9685 -- iter: 40/47
Training Step: 6000  | total loss: 0.11114 | time: 0.040s
| Adam | epoch: 1000 | loss: 0.11114 - acc: 0.9716 -- iter: 47/47
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [23]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [37]:
def chat(msg):
  #print("Start talking with the bot (type quit to stop)!")
  #while True:
    print(msg)
    #inp = input("You: ")
    inp = msg
    if inp.lower() == "quit":
        print("really worlking")

    results = model.predict([bag_of_words(inp, words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    if results[results_index] > 0.7:
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        return (random.choice(responses))
    else:
        answer="I don't understand so plz ask another question"
        return (answer)

In [ ]:
  def chat():
    print("Start talking with Pedro your AI assistant (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))
chat()

Start talking with Pedro your AI assistant (type quit to stop)!
Howdy Partner!
It's been a rough day but so far so good, how about yourslef?
Hi, how can I help?
Smoothies and Cookies!
The smoothie is $5 dollars and the cookie is $2 dolars
We do have great promotions! specially on mondays and tuesdays, buy one cookie get one free
We are open noon-11pm Monday-Friday! and Saturday 10am to midnight
My purpose is to give you information about our business but I have heard that you can meet people on dating apps these days!
Talk to you later amigo
Goodbye amigo!
Sad to see you go :(
Talk to you later amigo
My favorite color is Red!
